In [51]:


import pandas as pd
from sklearn.metrics.pairwise import linear_kernel

In [52]:
df = pd.read_csv('Seattle_Hotels.csv', encoding="latin-1")

In [53]:
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the..."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat..."
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the ..."
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...


In [54]:
len(df)

152

In [55]:
# 得到第1行的 二维数组
ex = df[df.index == 1][['name','desc']].values[0]
# print(ex['name'][1])
# print(type(ex))
print(ex[0])
# type(ex.values)

Sheraton Grand Seattle


In [56]:
ex

array(['Sheraton Grand Seattle',
       "Located in the city's vibrant core, the Sheraton Grand Seattle provides a gateway to the diverse sights and sounds of the Pacific Northwest. Step out of our front doors to find gourmet dining and bars, world-class shopping, exciting entertainment, and iconic local attractions including the Pike Place Market, Space Needle and Chihuly Garden & Glass Museum. As one of only seven Sheraton hotels in North America to earn the esteemed Grand designation, guests can book confidently knowing they?re receiving the highest benchmark on product and service offerings available. Experience our recently completed multimillion-dollar transformation featuring all new guest rooms, an expanded Sheraton Club Lounge, and modern meeting & event spaces. Gather in our stylish new lobby and enjoy our private art collection featuring local artists while enjoying your favorite beverage from Starbucks. The Sheraton Grand features several dining options including Loulay Kit

In [57]:
def get_description(index):
    # 获取第index行 的'name','desc'
    example = df[df.index == index][['desc','name']].values[0]
    if len(example) > 0 :
        print(example[0])
        print('Name:',example[1])

In [58]:
get_description(10)

Soak up the vibrant scene in the Living Room Bar and get in the mix with our live music and DJ series before heading to a memorable dinner at TRACE. Offering inspired seasonal fare in an award-winning atmosphere, it's a not-to-be-missed culinary experience in downtown Seattle. Work it all off the next morning at FIT®, our state-of-the-art fitness center before wandering out to explore many of the area's nearby attractions, including Pike Place Market, Pioneer Square and the Seattle Art Museum. As always, we've got you covered during your time at W Seattle with our signature Whatever/Whenever® service - your wish is truly our command.
Name: W Seattle


## 在删除停用词之前，可视化令牌（词汇）频率分布

统计每个词 出现的次数

In [59]:
import chart_studio.plotly as py
import plotly.graph_objs as go

In [60]:
# data = go.Scatter(x=[1, 2, 3], y=[4, 5, 6])
# fig = go.Figure(data=[data])
# # py.plot(fig, filename='simple-line')
# fig.show()

In [61]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer


# 传入预料数组Series  和 n  ,返回 前n个
def get_top_n_words(corpus_series,n=None):
    # 1.文本向量化
    vec = CountVectorizer().fit(corpus_series)
    # 2.运用 , 得到 稀疏矩阵 Coords(行列),Values
    bag_of_words = vec.transform(corpus_series)
    # 3.数据处理, 获取 词频数组 (词,出现总数)
    # print(bag_of_words)
   
    # 统计总数
    # 对稀疏矩阵 每一行求和,得到 每个特征字 的总数 [('',11)]
    sum_words = bag_of_words.sum(axis=0)
    
    # 将 词 和 总数 重新整理为[(词,总数)] , 通过idx匹配总数
    words_freq = [ (word,sum_words[0,idx]) for word , idx in vec.vocabulary_.items()]
    
    
    # 4.排序
    # words_freq.sort(key=lambda x:x[1],reverse=True)
    # 排序后 返回一个 新数组
    words_freq = sorted(words_freq,key=lambda x:x[1],reverse=True)
    # print(words_freq)
    return words_freq[:n]

# 1.调用获取词表方法
common_words = get_top_n_words(df['desc'],20)
# print(common_words)
# print(type(common_words))

# 2.词表 可视化
# 准备数据,list 转为 df
df1 = pd.DataFrame(common_words,columns=['words','count'])
# print(df1)

# 分组求和
words_count_series = df1.groupby('words').sum()['count'].sort_values()
# print(words_count_series)

# 渲染图表
fig = go.Figure(go.Bar(x=words_count_series.values,y=words_count_series.index, orientation='h'))
fig.update_layout(xaxis_title='Count',  # 设置 x 轴标题
    yaxis_title='Words', title_text='Top 20 words in hotel description before removing stop words', # 增加图表的高度
    height=500)
    

### Visualize Token (vocabulary) Frequency Distribution After Removing Stop Words
移除停用词后 的标记可视化分布

In [62]:
# 传入预料数组Series  和 n  ,返回 前n个
def get_top_n_words(corpus_series,n=None):
    # 1.文本向量化
    vec = CountVectorizer(stop_words='english').fit(corpus_series)
    # 2.运用 , 得到 稀疏矩阵 Coords(行列),Values
    bag_of_words = vec.transform(corpus_series)
    # 3.数据处理, 获取 词频数组 (词,出现总数)
    # print(bag_of_words)
   
    # 统计总数
    # 对稀疏矩阵 每一行求和,得到 每个特征字 的总数 [('',11)]
    sum_words = bag_of_words.sum(axis=0)
    
    # 将 词 和 总数 重新整理为[(词,总数)] , 通过idx匹配总数
    words_freq = [ (word,sum_words[0,idx]) for word , idx in vec.vocabulary_.items()]
    
    
    # 4.排序
    # words_freq.sort(key=lambda x:x[1],reverse=True)
    # 排序后 返回一个 新数组
    words_freq = sorted(words_freq,key=lambda x:x[1],reverse=True)
    # print(words_freq)
    return words_freq[:n]

# 1.调用获取词表方法
common_words = get_top_n_words(df['desc'],20)
# print(common_words)
# print(type(common_words))

# 2.词表 可视化
# 准备数据,list 转为 df
df1 = pd.DataFrame(common_words,columns=['words','count'])
# print(df1)

# 分组求和
words_count_series = df1.groupby('words').sum()['count'].sort_values()
# print(words_count_series)

# 渲染图表
fig = go.Figure(go.Bar(x=words_count_series.values,y=words_count_series.index, orientation='h'))
fig.update_layout(xaxis_title='Count',  # 设置 x 轴标题
    yaxis_title='Words', title_text='Top 20 words in hotel description before removing stop words', # 增加图表的高度
    height=500)
    

### Bigrams Frequency Distribution Before Removing Stop Word
二(n)元组的 词频分布

In [63]:
# 传入预料数组Series  和 n  ,返回 前n个
def get_top_n_words(corpus_series,n=None):
    # 1.文本向量化
    vec = CountVectorizer(stop_words='english',ngram_range=(2,2)).fit(corpus_series)
    # 2.运用 , 得到 稀疏矩阵 Coords(行列),Values
    bag_of_words = vec.transform(corpus_series)
    # 3.数据处理, 获取 词频数组 (词,出现总数)
    # print(bag_of_words)
   
    # 统计总数
    # 对稀疏矩阵 每一行求和,得到 每个特征字 的总数 [('',11)]
    sum_words = bag_of_words.sum(axis=0)
    
    # 将 词 和 总数 重新整理为[(词,总数)] , 通过idx匹配总数
    words_freq = [ (word,sum_words[0,idx]) for word , idx in vec.vocabulary_.items()]
    
    
    # 4.排序
    # words_freq.sort(key=lambda x:x[1],reverse=True)
    # 排序后 返回一个 新数组
    words_freq = sorted(words_freq,key=lambda x:x[1],reverse=True)
    # print(words_freq)
    return words_freq[:n]

# 1.调用获取词表方法
common_words = get_top_n_words(df['desc'],20)
# print(common_words)
# print(type(common_words))

# 2.词表 可视化
# 准备数据,list 转为 df
df1 = pd.DataFrame(common_words,columns=['words','count'])
# print(df1)

# 分组求和
words_count_series = df1.groupby('words').sum()['count'].sort_values()
# print(words_count_series)

# 渲染图表
fig = go.Figure(go.Bar(x=words_count_series.values,y=words_count_series.index, orientation='h'))
fig.update_layout(xaxis_title='Count',  # 设置 x 轴标题
    yaxis_title='Words', title_text='Top 20 words in hotel description before removing stop words', # 增加图表的高度
    height=500)
    

### Hotel Description Length Distribution

In [64]:
df['desc']

0      Located on the southern tip of Lake Union, the...
1      Located in the city's vibrant core, the Sherat...
2      Located in the heart of downtown Seattle, the ...
3      What?s near our hotel downtown Seattle locatio...
4      Situated amid incredible shopping and iconic a...
                             ...                        
147    Located in Queen Anne district, The Halcyon Su...
148    Just a block from the world famous Space Needl...
149    Stay Alfred on Wall Street resides in the hear...
150    The perfect marriage of heightened convenience...
151    Yes, it's true. Every room at citizenM is the ...
Name: desc, Length: 152, dtype: object

In [65]:
# 作为新列
df['desc_word_count'] = df['desc'].apply(lambda x:len(str(x).split()))

In [66]:
import numpy as np
# 描述字数统计
desc_lengths = list(df['desc_word_count'])

print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Number of descriptions: 152 
Average word count 156.94736842105263 
Minimum word count 16 
Maximum word count 494


In [67]:
# 描述信息字数 统计
word_counts = df['desc_word_count']

# 创建一个新的 Figure 对象
# fig = go.Figure()

# 添加直方图的 
fig = go.Figure(go.Histogram(
    x=word_counts,  # 直方图的数据
    nbinsx=50,      # x轴上的bins数量
    # linecolor='black',  # 直方图线条颜色
    # opacity=0.75  # 直方图的透明度
))

# 更新布局
fig.update_layout(
    xaxis_title='Word Count',  # x轴标题
    yaxis_title='Count',       # y轴标题
    title_text='Word Count Distribution in Hotel Description'  # 图表标题
)

# 显示图表
# fig.show()

### Preprocessing hotel description text

In [68]:
import nltk
# from nltk.corpus import stopwords
# 查看nltk_data，停用词 保存路径
print(nltk.find(''))

D:\nltk_data


#### 数据预处理

In [69]:

from nltk.corpus import stopwords
import re

REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')

# 确保已经下载了停用词数据集
# nltk.download('stopwords')

STOPWORDS = set(stopwords.words('english'))
# print(len(STOPWORDS))
def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['desc_clean'] = df['desc'].apply(clean_text)
# clean后 字数对比
print('clean一些停用词后 字数对比:')
print(len(df['desc_clean'].iloc[0]))
print(len(df['desc'].iloc[0]))

clean一些停用词后 字数对比:
892
1173


##### 输出 指定 i 行 数据

In [70]:
# 输出 指定 i 行 数据
def print_desc_clean(i):
    ex = df[df.index == i][['desc_clean','name']].values[0]
    if len(ex) > 0:
        # print(type(ex))
        print('desc:',ex[0])
        print('name:',ex[1])
        
print_desc_clean(30)

desc: hotel andra stylish 119 guestroom boutique luxury hotel ideally located heart downtown seattle nestled south lake union belltown next retail core surrounding downtown area thriving food music art history within walking distance famous pike place market seattle center space needle hotel andra provides sophisticated urban escape firstclass luxury hotel offerings hotel ndra also offers sophisticated spaces meetings events venues northern lights room galleri ndra loft would love help arrange celebrations kind private dinners board meetings events events done together exclusive catering tom douglas seattles celebrated chef restauranteur
name: Hotel Andra


#### 模型设计

In [71]:
# 1.df中name设为索引

# 2.生成 相似度矩阵

# 3.将name索引转为series

# 4.构造方法 ,输入 【name 和 相似度矩阵】,输出 【推荐数组】
#     1.根据name获得 矩阵里对应行的 相似度，
#     2.再转为series 排序，
#     3.通过 索引对应name 找到 原表对应的 酒店名
#     4.加入数组
#     5.输出数组

In [72]:
df.set_index('name',inplace=True)

In [73]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0.0,stop_words='english')
# 运用,  坐标+值的 矩阵
tfidf_matrix = tf.fit_transform(df['desc_clean'])
# 矩阵 每对之间的 （线性核）余弦相似度
cosine_similarities = linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_similarities

array([[1.        , 0.01161917, 0.02656894, ..., 0.01184587, 0.00244782,
        0.00583589],
       [0.01161917, 1.        , 0.015586  , ..., 0.01625083, 0.00313105,
        0.00797999],
       [0.02656894, 0.015586  , 1.        , ..., 0.02071479, 0.00748781,
        0.01028037],
       ...,
       [0.01184587, 0.01625083, 0.02071479, ..., 1.        , 0.01066904,
        0.0079114 ],
       [0.00244782, 0.00313105, 0.00748781, ..., 0.01066904, 1.        ,
        0.00257955],
       [0.00583589, 0.00797999, 0.01028037, ..., 0.0079114 , 0.00257955,
        1.        ]])

In [74]:
indices = pd.Series(df.index)

In [75]:
indices[0:20]

0              Hilton Garden Seattle Downtown
1                      Sheraton Grand Seattle
2               Crowne Plaza Seattle Downtown
3               Kimpton Hotel Monaco Seattle 
4                          The Westin Seattle
5                 The Paramount Hotel Seattle
6                              Hilton Seattle
7                               Motif Seattle
8                             Warwick Seattle
9                  Four Seasons Hotel Seattle
10                                  W Seattle
11                         Gand Hyatt Seattle
12                       Kimpton Alexis Hotel
13                                  Hotel Max
14                          Ace Hotel Seattle
15                Seattle Marriott Waterfront
16                The Edgewater Hotel Seattle
17         SpringHill Suites Seattle Downtown
18                     Fairmont Olympic Hotel
19    La Quinta Inn & Suites Seattle Downtown
Name: name, dtype: object

#### 获取name对应行 的相似度

In [79]:
# 通过name 获取 对应行索引
indices[indices == 'Hilton Seattle'].index[0]


np.int64(6)

In [80]:
# 通过 行索引 在相似度矩阵中 找到对应的 行数据
cosine_similarities[6]

array([0.01899395, 0.03444136, 0.01562442, 0.03795618, 0.03564992,
       0.04097353, 1.        , 0.01425689, 0.01349041, 0.0113087 ,
       0.03248299, 0.01118827, 0.00685692, 0.00406017, 0.00262039,
       0.01831067, 0.0203859 , 0.02335899, 0.01123169, 0.03543304,
       0.0070664 , 0.0036242 , 0.02763601, 0.03426073, 0.00965453,
       0.03069243, 0.00943533, 0.0084885 , 0.03155077, 0.01182099,
       0.01696738, 0.02146152, 0.0344257 , 0.01227887, 0.03903306,
       0.08071003, 0.00197544, 0.011409  , 0.04000142, 0.01889037,
       0.00859497, 0.0058285 , 0.00970696, 0.02018602, 0.02776597,
       0.00396098, 0.02544213, 0.00984421, 0.00661904, 0.00657421,
       0.01324211, 0.01607117, 0.01318813, 0.00626402, 0.02748449,
       0.03981335, 0.01961058, 0.02033834, 0.01330426, 0.01131197,
       0.02652243, 0.01857435, 0.01051124, 0.0136229 , 0.00811548,
       0.01276071, 0.02902152, 0.00578514, 0.00605646, 0.00569269,
       0.03038125, 0.01948586, 0.00677777, 0.02429242, 0.03395

In [99]:
def recommendations(name,cosine_similarities = cosine_similarities):
    recommended_hotels =[]
    name_index = indices[indices == name].index[0]
    name_similarities = cosine_similarities[name_index]
    asc_name_series = pd.Series(name_similarities).sort_values(ascending=False)
    # asc_name_series.iloc[0:10] ,第0项为自己
    asc_name_index = asc_name_series.iloc[1:11].index
    asc_name_index_list = list(asc_name_index)
    # 获得 相似酒店 序号索引
    # print(asc_name_index_list)
#     根据 序号 找酒店
    for i in asc_name_index_list:
        # print(i)
        # print(list(df.index))
        # 【nameIndex 转为 list 再选 第 i 个 name】
        top10_nameIndex_str = list(df.index)[i]
        # print(top10_nameIndex_str)
        recommended_hotels.append(top10_nameIndex_str)
    return recommended_hotels
    
    
recommendations('Hilton Seattle')

['Hyatt Regency Seattle',
 'Residence Inn by Marriott Seattle Downtown/Lake Union',
 'Quality Inn & Suites Seattle Center',
 'Homewood Suites by Hilton Seattle Downtown',
 'Hotel Seattle',
 'The Paramount Hotel Seattle',
 'Holiday Inn Express & Suites Seattle-City Center',
 'Clarion Hotel Seattle Airport',
 'Homewood Suites by Hilton Seattle Convention Center Pike Street',
 'Crown Inn Motel']